In [39]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.basemap import Basemap

In [40]:
# read in data from July, 28th, 2012
trajectories28 = pd.read_csv("TrajDataV2_20120728.txt",  
                             delimiter="\t", 
                             names=('flight', 'date', 'wind', 'time', 'speed', 'altitude','latitude', 'longitude', 'nan'),
                             usecols=('flight', 'date', 'wind', 'time', 'speed', 'altitude','latitude', 'longitude')
                            )
# read in data from July, 29th, 2012
trajectories29 = pd.read_csv("TrajDataV2_20120729.txt",  
                             delimiter="\t", 
                             names=('flight', 'date', 'wind', 'time', 'speed', 'altitude','latitude', 'longitude', 'nan'),
                             usecols=('flight', 'date', 'wind', 'time', 'speed', 'altitude','latitude', 'longitude')
                            )
trajectories28.head()

,flight,date,wind,time,speed,altitude,latitude,longitude
0,845,28,0,1,470.31621,39000,40.692500,-74.168667
1,845,28,0,2,470.31621,39000,40.784686,-74.019193
2,845,28,0,3,470.31621,39000,40.877281,-73.868705
3,845,28,0,4,470.31621,39000,40.969822,-73.717979
4,845,28,0,5,470.31621,39000,41.062186,-73.566848


In [78]:
# concatenate data from both days
#trajectories = pd.concat([trajectories28, trajectories29])
trajectories = trajectories29

In [79]:
# add consecutive flight index to the data
flightNames = trajectories['flight'].unique()
trajectories['flightIndex'] = trajectories['flight'].map(lambda x: np.where(flightNames==x)[0][0])
trajectories.head()

,flight,date,wind,time,speed,altitude,latitude,longitude,flightIndex
0,805,29,0,2,465.28117,33000,40.692500,-74.168667,0
1,805,29,0,3,465.28117,33000,40.785863,-74.013012,0
2,805,29,0,4,465.28117,33000,40.878762,-73.856473,0
3,805,29,0,5,465.28117,33000,40.971273,-73.699485,0
4,805,29,0,6,465.28117,33000,41.063590,-73.542087,0


In [80]:
# set consecutive flight index as dataset index
trajectories = trajectories.set_index('flightIndex')
trajectories.head()

,flight,date,wind,time,speed,altitude,latitude,longitude
flightIndex,,,,,,,,
0,805,29,0,2,465.28117,33000,40.692500,-74.168667
0,805,29,0,3,465.28117,33000,40.785863,-74.013012
0,805,29,0,4,465.28117,33000,40.878762,-73.856473
0,805,29,0,5,465.28117,33000,40.971273,-73.699485
0,805,29,0,6,465.28117,33000,41.063590,-73.542087


In [86]:
trajectories.time.plot(marker='o', linestyle='o')
plt.show()

In [83]:
def plotTrajectories(trajectories):
    # Create a figure of size (i.e. pretty big)
    fig = plt.figure(figsize=(20,10))

    # Create a map, using the Gall–Peters projection, 
    map = Basemap(projection='gall', 
                  # with low resolution,
                  resolution = 'l', 
                  # And threshold 100000
                  area_thresh = 100000.0,
                  # Centered at 0,0 (i.e null island)
                  lat_0=0, lon_0=0)

    # Draw the coastlines on the map
    map.drawcoastlines()

    # Draw country borders on the map
    map.drawcountries()

    # Fill the land with grey
    map.fillcontinents(color = '#888888')

    # Draw the map boundaries
    map.drawmapboundary(fill_color='#f4f4f4')

    # Define our longitude and latitude points
    # We have to use .values because of a wierd bug when passing pandas data
    # to basemap.
    x,y = map(trajectories['longitude'].values, trajectories['latitude'].values)

    # Plot them using round markers of size 6
    map.plot(x, y, 'b', markersize=6)

    # Show the map
    plt.show()

In [84]:
plotTrajectories(trajectories[trajectories.index < 100])